## MatchEd Analysis: Control Condition

This notebook examines data collected in condition ID 660af3a006227e71ec8eb992, which is the control arm intended to be a direct replication of Chen and Sönmez (2006). Reads data from the corresponding condition CSV file. Then, it enriches the data with bias classifications according to the 2006 paper. We look at rates of truthful preference revelation and patterns in the biases to compare to the original paper.

## Processing and Basic Analysis

Conduct some basic analysis on truth-telling behavior and classify preference manipulations into different types of bias. Pivots CSV data from 1 row per student-school combination into one row per student with columns for each submitted and truthful ranking. This should enable easier analysis for something like a logit regression later. Data output into file `control_processed.csv`.

In [181]:
import pandas as pd
import numpy as np

In [182]:
# Read in the data
obj_id = "660af3a006227e71ec8eb992"
control_data = pd.read_csv(f'condition_{obj_id}.csv')
control_data.head()

,student_id,school_id,truthful_rank,payoff,submitted_rank,is_district
0,1,0,2,13,5.0,True
1,1,1,1,16,1.0,False
2,1,2,4,9,2.0,False
3,1,3,7,2,7.0,False
4,1,4,6,5,3.0,False


In [183]:
# We didn't get data for all students. Get a list of ids with data, meaning submitted ran is not null
ids_with_data = control_data[control_data['submitted_rank'].notnull()]['student_id'].unique()
print("We got data for", len(ids_with_data), "students with ids", ids_with_data)

We got data for 28 students with ids [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 34 36]


In [184]:
# Filter out those rows where submitted_rank is empty, and cast submitted rank to int
control_data = control_data.loc[control_data['submitted_rank'].notnull()]
control_data['submitted_rank'] = control_data['submitted_rank'].astype(int)

In [185]:
control_data['correct'] = control_data['submitted_rank'] == control_data['truthful_rank']
# Group by student id and return number of students who got all correct
grouped_students = control_data.groupby('student_id')['correct'].sum()
correct_students = grouped_students[grouped_students == 7]
# Add a column to the dataframe to indicate if the student is correct
control_data['complete_truthtell'] = control_data['student_id'].isin(correct_students.index)
print("The number of students who ranked all preferences completely truthfully is", correct_students.shape[0], "out of", control_data['student_id'].nunique(), ". The ids are:", correct_students.index.values)

The number of students who ranked all preferences completely truthfully is 8 out of 28 . The ids are: [ 7 15 16 17 18 19 24 25]


Of the results collected, a very small percentage reported entirely truthfully. We only observe 7 total participants reporting truthfully. We will first qualify this result by following Chen and Sönmez's method: "Under GS, truthful preference revelation requires that the reported ranking is identical to the true preference ranking from the first choice up to the participant's district school. The remaining rankings, from the district school to the last choice, are irrelevant under both GS and TTC." We are also interested in seeing if having your district school as your first preference predicts truthful preference revelation (as we hypothesize it should). There are only two instances in the data where the district school is also the top choice. In our results, 1 of the 2 participants reported truthfully. This is obviously non-optimal play because the DA algorithm guarantees participants a school at least as desirable as their district school.

In [186]:
groups = control_data.groupby('student_id')
count = 0
chen_sonmez_truthful = []
for _, group in groups:
    sorted_group = group.sort_values('truthful_rank')
    district_index = sorted_group[sorted_group['is_district']].index[0]

    # Check if series are equal up to and including the district index
    if (sorted_group.loc[:district_index, 'truthful_rank'] == sorted_group.loc[:district_index, 'submitted_rank']).all():
        chen_sonmez_truthful.append(group['student_id'].iloc[0])
        count += 1

# Print the result
print("Number of people with truthful_rank = submitted_rank up to and including the the district school:", count, "out of", control_data['student_id'].nunique(), ", yielding a percentage of", count/control_data['student_id'].nunique() * 100, "%", ". The ids are:", chen_sonmez_truthful)
control_data['partial_truthtell'] = control_data['student_id'].isin(chen_sonmez_truthful)
control_data.head(8)

Number of people with truthful_rank = submitted_rank up to and including the the district school: 10 out of 28 , yielding a percentage of 35.714285714285715 % . The ids are: [2, 7, 10, 15, 16, 17, 18, 19, 24, 25]


,student_id,school_id,truthful_rank,payoff,submitted_rank,is_district,correct,complete_truthtell,partial_truthtell
0,1,0,2,13,5,True,False,False,False
1,1,1,1,16,1,False,True,False,False
2,1,2,4,9,2,False,False,False,False
3,1,3,7,2,7,False,True,False,False
4,1,4,6,5,3,False,False,False,False
5,1,5,3,11,4,False,False,False,False
6,1,6,5,7,6,False,False,False,False
7,2,0,1,16,1,True,True,False,True


In [187]:
# Does having district school as top choice predict truthful preference revelation?
district_school_top_choice = []
for _, group in groups:
    district_index = group[group['is_district']].index[0]
    if group.loc[district_index, 'truthful_rank'] == 1:
        district_school_top_choice.append(group['student_id'].iloc[0])

print("Following student IDs had district school as their highest payoff school/truthful top choice:", district_school_top_choice, ". The subset that also ranked all preferences truthfully up to and including the district school is:", set(district_school_top_choice).intersection(set(chen_sonmez_truthful)))

# Respondent 5 didn't rank district school as top choice even though it had the highest payoff (quite counterintuitive)
groups.get_group(5).sort_values('truthful_rank')
    

Following student IDs had district school as their highest payoff school/truthful top choice: [2, 5] . The subset that also ranked all preferences truthfully up to and including the district school is: {2}


,student_id,school_id,truthful_rank,payoff,submitted_rank,is_district,correct,complete_truthtell,partial_truthtell
29,5,1,1,16,3,True,False,False,False
32,5,4,2,13,1,False,False,False,False
28,5,0,3,11,2,False,False,False,False
34,5,6,4,9,4,False,True,False,False
33,5,5,5,7,5,False,True,False,False
31,5,3,6,5,6,False,True,False,False
30,5,2,7,2,7,False,True,False,False


We also want to look at factors that predict truthful preference revelation and the effects of district schools and small schools (Chen and Sönmez conduct a logit regression estimation of the effects of small and district school on truthful preference revelation.). They divide preference manipulation into the mutually exclusive categories of:

* district school bias $\rightarrow$ A participant puts her district school into a higher position than that in the true preference order.
* small school bias $\rightarrow$ A participant puts school A or B (or both) into lower positions than those in the true preference ordering.
* similar preferences bias $\rightarrow$ A participant puts schools with the highest payoffs into lower positions.

In [188]:
# Recall that putting district school as higher choice means the submitted rank is better (ie. lower number) than the truthful rank
district_school_bias_index = []
small_school_bias_index = []
similar_preference_index = []
for i, group in groups:
    district_index = group[group['is_district']].index[0]
    # Check for district school bias
    if group.loc[district_index, 'truthful_rank'] > group.loc[district_index, 'submitted_rank']:
        district_school_bias_index.append(group['student_id'].iloc[0])

    # Check for small school bias by comparing truthful and submitted rank for school ids 0 and 1 (already sorted by school_id)
    first_row_idx = (i - 1) * group.shape[0]
    if group.loc[first_row_idx, 'truthful_rank'] < group.loc[first_row_idx, 'submitted_rank'] or group.loc[first_row_idx + 1, 'truthful_rank'] < group.loc[first_row_idx + 1, 'submitted_rank']:
        small_school_bias_index.append(group['student_id'].iloc[0])
    
    # Check for similar preferences bias (push highest payoff lower b/c fear of competition)
    group = group.sort_values('payoff', ascending=False)
    first_row = group.head(1)
    if first_row.iloc[0]['truthful_rank'] < first_row.iloc[0]['submitted_rank']:
        similar_preference_index.append(group['student_id'].iloc[0])

# Print results:
print("Following student IDs had district school bias:", district_school_bias_index)
print("Following student IDs had small school bias:", small_school_bias_index)
print("Following student IDs had similar preference bias:", similar_preference_index)
print("Following student IDs had partial truthful preference revelation:", chen_sonmez_truthful)

Following student IDs had district school bias: [3, 6, 8, 11, 12, 13, 14, 34, 36]
Following student IDs had small school bias: [1, 3, 4, 5, 6, 8, 9, 10, 13, 14, 20, 21, 23, 26, 34, 36]
Following student IDs had similar preference bias: [3, 4, 5, 6, 8, 9, 13, 14, 20, 21, 26, 34, 36]
Following student IDs had partial truthful preference revelation: [2, 7, 10, 15, 16, 17, 18, 19, 24, 25]


In [189]:
# Add binary columns to the dataframe to indicate if the student has the respective bias
control_data['dsb'] = control_data['student_id'].isin(district_school_bias_index)
control_data['ssb'] = control_data['student_id'].isin(small_school_bias_index)
control_data['spb'] = control_data['student_id'].isin(similar_preference_index)
control_data.head(8)

,student_id,school_id,truthful_rank,payoff,submitted_rank,is_district,correct,complete_truthtell,partial_truthtell,dsb,ssb,spb
0,1,0,2,13,5,True,False,False,False,False,True,False
1,1,1,1,16,1,False,True,False,False,False,True,False
2,1,2,4,9,2,False,False,False,False,False,True,False
3,1,3,7,2,7,False,True,False,False,False,True,False
4,1,4,6,5,3,False,False,False,False,False,True,False
5,1,5,3,11,4,False,False,False,False,False,True,False
6,1,6,5,7,6,False,False,False,False,False,True,False
7,2,0,1,16,1,True,True,False,True,False,False,False


Note: "In the designed environment, as A and B are ranked highly by all agents, upgrading the district school in most cases implies a downgrading of school A or B." Hence, DSB and SSB often come hand in hand.

In [190]:
# Add columns for DsbSSb, DsbSPb, SsbSPb, DsbSSbSPb
control_data['DsbSSb'] = control_data['dsb'] & control_data['ssb']
control_data['DsbSPb'] = control_data['dsb'] & control_data['spb']
control_data['SsbSPb'] = control_data['ssb'] & control_data['spb']
control_data['DsbSSbSPb'] = control_data['dsb'] & control_data['ssb'] & control_data['spb']
control_data['DsbOnly'] = control_data['dsb'] & ~control_data['ssb'] & ~control_data['spb']
control_data.head(8)

,student_id,school_id,truthful_rank,payoff,submitted_rank,is_district,correct,complete_truthtell,partial_truthtell,dsb,ssb,spb,DsbSSb,DsbSPb,SsbSPb,DsbSSbSPb,DsbOnly
0,1,0,2,13,5,True,False,False,False,False,True,False,False,False,False,False,False
1,1,1,1,16,1,False,True,False,False,False,True,False,False,False,False,False,False
2,1,2,4,9,2,False,False,False,False,False,True,False,False,False,False,False,False
3,1,3,7,2,7,False,True,False,False,False,True,False,False,False,False,False,False
4,1,4,6,5,3,False,False,False,False,False,True,False,False,False,False,False,False
5,1,5,3,11,4,False,False,False,False,False,True,False,False,False,False,False,False
6,1,6,5,7,6,False,False,False,False,False,True,False,False,False,False,False,False
7,2,0,1,16,1,True,True,False,True,False,False,False,False,False,False,False,False


In [191]:
# Store intermediate results, collapsing the dataframe to one row per student to make for easier analysis by pivoting
new_data = control_data.groupby('student_id').agg({
    'complete_truthtell': 'first',
    'partial_truthtell': 'first',
    'dsb': 'first',
    'ssb': 'first',
    'spb': 'first',
    'DsbSSb': 'first',
    'DsbSPb': 'first',
    'SsbSPb': 'first',
    'DsbSSbSPb': 'first',
    'DsbOnly': 'first'
}).reset_index()
print(new_data.shape)
print(len(control_data[control_data['truthful_rank'] == 6].groupby('student_id')['school_id'].first().values))
# Append 7 columns for r1-r7, indicating the school ID for each of the 7 submitted rankings
for i in range(1, 8):
    new_data[f'r{i}'] = control_data[control_data['submitted_rank'] == i].groupby('student_id')['school_id'].first().values

# Append 7 columns for t1-t7, indicating the school ID for each of the 7 truthful rankings
for i in range(1, 8):
    new_data[f't{i}'] = control_data[control_data['truthful_rank'] == i].groupby('student_id')['school_id'].first().values

# Append 7 columns for p1-p7, indicating the payoff for each of the school Ids 0 to 6
for i in range(7):
    new_data[f'p{i}'] = control_data[control_data['school_id'] == i].groupby('student_id')['payoff'].first().values

new_data.head()


(28, 11)
28


,student_id,complete_truthtell,partial_truthtell,dsb,ssb,spb,DsbSSb,DsbSPb,SsbSPb,DsbSSbSPb,...,t5,t6,t7,p0,p1,p2,p3,p4,p5,p6
0,1,False,False,False,True,False,False,False,False,False,...,6,4,3,13,16,9,2,5,11,7
1,2,False,True,False,False,False,False,False,False,False,...,3,5,4,16,13,11,7,2,5,9
2,3,False,False,True,True,True,True,True,True,True,...,2,6,4,11,13,7,16,2,9,5
3,4,False,False,False,True,True,False,False,True,False,...,5,3,4,16,13,11,5,2,7,9
4,5,False,False,False,True,True,False,False,True,False,...,5,3,2,11,16,2,5,13,7,9


In [192]:
# Store the final results
new_data.to_csv(f'control_processed.csv', index=False)

# Probabilistic Model of Truthtelling Behavior